In [ ]:
from bs4 import BeautifulSoup
import requests
import csv
import time

In [ ]:
filecsv = open('imdb10.csv', 'w', encoding='utf8')
csv_columns = ['Url', 'Title', 'Year' ,'Rate', 'Votes', 'Genre', 'Certificate', 'Duration', 'People', 'Nudity', 'Violence', 'Profanity', 'Alcohol', 'Frightening']
writer = csv.DictWriter(filecsv, fieldnames=csv_columns, delimiter=';')
writer.writeheader()

x = 1
s = requests.Session()

headers = {'Accept-Language': 'en-US, en;q=0.5'}

In [ ]:
for page in range(1,10000, 50):
  url = f'https://www.imdb.com/search/title/?title_type=feature&adult=include&sort=boxoffice_gross_us,desc&start={page}'
  req = requests.get(url)
  soup = BeautifulSoup(req.content, 'html.parser')
  data = soup.find_all('div', {'class':'lister-item mode-advanced'})

  for film in data:     
    title = film.find('h3', {'class':'lister-item-header'}).find('a').text
    
    year = film.find('span', {'class':'lister-item-year text-muted unbold'}).text.replace('(', '').replace(')', '').strip()
    for i in year.split():
       if i.isdigit():
         year = i

    rate = film.find('strong')
    if rate:
      rate = rate.text
    else:
      rate = 'No Rate'

    vote = film.find('span', {'name':'nv'})
    if vote:
      votes = vote.text
    else:
      votes = 'No Votes'

    
    genre = film.find('span', {'class':'genre'})
    if genre:
      genre = genre.text.strip()
    else:
      genre = 'No Genre'

    cert = film.find_all('span', {'class':'certificate'})
    if cert:
      certificate = cert[0].text
    else:
      certificate = 'None'

    duration = film.find('span', {'class':'runtime'})
    if duration:
      duration = duration.text[:-3]
    else:
      duration = 'None'
    
    people = film.find_all('p', {'class':''})
    if people:
      people = people[1].text
    else:
      people = 'None'
    
    link_url = film.find('h3', {'class':'lister-item-header'}).find('a').attrs['href']  
    url2 = 'https://www.imdb.com'+link_url+'parentalguide'
    req2 = requests.get(url2)
    soup2 = BeautifulSoup(req2.content, 'html.parser')
 
    if soup2 == 'None':
      nudity = 'No Rate'
      violence = 'No Rate'
      profanity = 'No Rate'
      alcohol = 'No Rate'
      frightening = 'No Rate'
    else:
      element = soup2.find_all('div', {'class':'advisory-severity-vote__container ipl-zebra-list__item'})
      if element:
        nudity = element[0].find('span')
        if nudity:
          nudity = nudity.text
        else:
          nudity = 'No Rate'

        violence = element[2].find('span')
        if violence:   
          violence = violence.text
        else:
          violence = 'No Rate'

        profanity = element[4].find('span')
        if profanity:
          profanity = profanity.text
        else:
          profanity = 'No Rate'

        alcohol = element[6].find('span')
        if alcohol:    
          alcohol = alcohol.text
        else:
          alcohol = 'No Rate'

        frightening = element[8].find('span')
        if frightening:
          frightening = frightening.text
        else:
          frightening = 'No Rate'

    writer.writerow({'Url':link_url, 'Title':title, 'Year':year ,'Rate':rate, 'Votes':votes, 'Genre':genre, 'Certificate':certificate, 
                     'Duration':duration, 'People':people, 'Nudity':nudity, 'Violence':violence, 
                     'Profanity':profanity, 'Alcohol':alcohol, 'Frightening':frightening})